#Part 2

## Part 2.1

### Data Preparation + Cleaning

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn import feature_selection, linear_model, model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
# read csv (google collab)
data_url = "/content/gdrive/MyDrive/UNI/Big Data Problem/Manhattan12.csv"
df = pd.read_csv(data_url, header = 4)
df = df.iloc[18:] #delete first 18 rows due to lack of data
df.shape

(27377, 21)

In [ ]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APART\nMENT\nNUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE\nPRICE,SALE DATE
18,1,ALPHABET CITY,03 THREE FAMILY HOMES,1,377,66,,C0,243 EAST 7TH STREET,,...,3,0,3,"2,381","3,084",1899,1,C0,"$3,150,000",06/03/2013
19,1,ALPHABET CITY,04 TAX CLASS 1 CONDOS,1C,399,1102,,R6,238 EAST 4TH STREET,-,...,1,0,1,0,0,1955,1,R6,"$3,450,000",27/03/2013
20,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,374,1,,C7,303 EAST 4TH STREET,,...,8,2,10,"1,501","6,929",1900,2,C7,$0,05/02/2013
21,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,375,62,,C4,715 EAST 5TH STREET,,...,20,0,20,"2,426","9,345",1900,2,C4,"$3,650,000",06/09/2012
22,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,376,30,,C4,274 EAST SEVENTH STREET,,...,13,0,13,"2,726","13,002",1910,2,C4,"$895,250",25/10/2012


In [ ]:
df = df.rename(columns={"SALE\nPRICE":"SALE PRICE", "APART\nMENT\nNUMBER":"APARTMENT NUMBER"}) #rename columns

In [ ]:
df = df.drop(columns=['BOROUGH', 'EASE-MENT', 'APARTMENT NUMBER'])

In [ ]:
def price_to_int(column):
    df[column] = df[column].str.replace(',','', regex=True).str.replace('$','', regex=True).astype('int')
  
price_to_int('SALE PRICE')
price_to_int('LAND SQUARE FEET')
price_to_int('GROSS SQUARE FEET')
price_to_int('TOTAL UNITS')
price_to_int('RESIDENTIAL UNITS')

In [ ]:
# list all numerical variables 
df_nums = df.select_dtypes(include=[np.number])
list(df_nums)

['BLOCK',
 'LOT',
 'ZIP CODE',
 'RESIDENTIAL UNITS',
 'COMMERCIAL UNITS',
 'TOTAL UNITS',
 'LAND SQUARE FEET',
 'GROSS SQUARE FEET',
 'YEAR BUILT',
 'TAX CLASS AT TIME OF SALE',
 'SALE PRICE']

In [ ]:
# list all categorical values
df_cat = df.select_dtypes(include=[object])
list(df_cat)

['NEIGHBORHOOD',
 'BUILDING CLASS CATEGORY',
 'TAX CLASS AT PRESENT',
 'BUILDING CLASS AT PRESENT',
 'ADDRESS',
 'BUILDING CLASS AT TIME OF SALE',
 'SALE DATE']

In [ ]:
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'], dayfirst=True) # convert sale date to datetime

In [ ]:
# replace 0 with NaN
df.replace(0, np.nan, inplace=True) 
df.replace("0", np.nan, inplace=True)

pd.isnull(df).sum()

NEIGHBORHOOD                          0
BUILDING CLASS CATEGORY               0
TAX CLASS AT PRESENT                  0
BLOCK                                 0
LOT                                   0
BUILDING CLASS AT PRESENT             0
ADDRESS                               0
ZIP CODE                              1
RESIDENTIAL UNITS                 16354
COMMERCIAL UNITS                  23944
TOTAL UNITS                        9687
LAND SQUARE FEET                  22888
GROSS SQUARE FEET                 23051
YEAR BUILT                         6424
TAX CLASS AT TIME OF SALE             0
BUILDING CLASS AT TIME OF SALE        0
SALE PRICE                         7593
SALE DATE                             0
dtype: int64

In [ ]:
# finding out how many dupliactes there are by subtracting duplicates from 'drop duplicates' from entire dataframe
len(df)-len(df.drop_duplicates())

1593

In [ ]:
df.drop_duplicates(keep='first', inplace=True)
df = df[df['ZIP CODE'].notna()] #there was 1 missing na value

df.shape

(25783, 18)

In [ ]:
pd.isnull(df).sum()

NEIGHBORHOOD                          0
BUILDING CLASS CATEGORY               0
TAX CLASS AT PRESENT                  0
BLOCK                                 0
LOT                                   0
BUILDING CLASS AT PRESENT             0
ADDRESS                               0
ZIP CODE                              0
RESIDENTIAL UNITS                 14996
COMMERCIAL UNITS                  23125
TOTAL UNITS                        9617
LAND SQUARE FEET                  22119
GROSS SQUARE FEET                 22268
YEAR BUILT                         5826
TAX CLASS AT TIME OF SALE             0
BUILDING CLASS AT TIME OF SALE        0
SALE PRICE                         6872
SALE DATE                             0
dtype: int64

In [ ]:
#Using linear interpolation to fill in the missing gaps in all the data.
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].interpolate(method ='linear', limit_direction ='both')
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].interpolate(method ='linear', limit_direction ='both')
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].interpolate(method ='linear', limit_direction ='both')
df['YEAR BUILT'] = df['YEAR BUILT'].interpolate(method ='linear', limit_direction ='both')
df['COMMERCIAL UNITS'] = df['COMMERCIAL UNITS'].interpolate(method ='linear', limit_direction ='both')
df['RESIDENTIAL UNITS'] = df['RESIDENTIAL UNITS'].interpolate(method ='linear', limit_direction ='both')
df['SALE PRICE'] = df['SALE PRICE'].interpolate(method ='linear', limit_direction ='both')
df["BUILDING CLASS CATEGORY"] = df["BUILDING CLASS CATEGORY"].interpolate(method='pad')

pd.isnull(df).sum()

<ipython-input-422-685555a3c8a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].interpolate(method ='linear', limit_direction ='both')
<ipython-input-422-685555a3c8a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].interpolate(method ='linear', limit_direction ='both')
<ipython-input-422-685555a3c8a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

NEIGHBORHOOD                         0
BUILDING CLASS CATEGORY              0
TAX CLASS AT PRESENT                 0
BLOCK                                0
LOT                                  0
BUILDING CLASS AT PRESENT            0
ADDRESS                              0
ZIP CODE                             0
RESIDENTIAL UNITS                    0
COMMERCIAL UNITS                     0
TOTAL UNITS                       9617
LAND SQUARE FEET                     0
GROSS SQUARE FEET                    0
YEAR BUILT                           0
TAX CLASS AT TIME OF SALE            0
BUILDING CLASS AT TIME OF SALE       0
SALE PRICE                           0
SALE DATE                            0
dtype: int64

In [ ]:
def convert_to_int(column): #Converts columns to the integer type
  df[column] = df[column].apply(int)

columns_to_int = ["COMMERCIAL UNITS", "RESIDENTIAL UNITS", "LAND SQUARE FEET", "GROSS SQUARE FEET", "YEAR BUILT", "ZIP CODE"]

for x in columns_to_int:
  convert_to_int(x)

In [ ]:
df.dtypes

NEIGHBORHOOD                              object
BUILDING CLASS CATEGORY                   object
TAX CLASS AT PRESENT                      object
BLOCK                                      int64
LOT                                        int64
BUILDING CLASS AT PRESENT                 object
ADDRESS                                   object
ZIP CODE                                   int64
RESIDENTIAL UNITS                          int64
COMMERCIAL UNITS                           int64
TOTAL UNITS                              float64
LAND SQUARE FEET                           int64
GROSS SQUARE FEET                          int64
YEAR BUILT                                 int64
TAX CLASS AT TIME OF SALE                  int64
BUILDING CLASS AT TIME OF SALE            object
SALE PRICE                               float64
SALE DATE                         datetime64[ns]
dtype: object

In [ ]:
df['TOTAL UNITS'] = df['RESIDENTIAL UNITS'] + df['COMMERCIAL UNITS'] #Add the residential and commercial units together